In [1]:
import os
import cv2
import random

In [2]:
base_path = r"D:\Datasets\Celeb df\Original dataset\Celeb DF(V2)"
celeb_real_path      = os.path.join(base_path, "Celeb-real")
youtube_real_path    = os.path.join(base_path, "YouTube-real")
celeb_synthesis_path = os.path.join(base_path, "Celeb-synthesis")

In [3]:
def make_list_of_videos_path(folder_path):
    videos_list = [os.path.join(folder_path, video_file_name) for video_file_name in os.listdir(folder_path)]
    return videos_list

celeb_synthesis_videos_path = make_list_of_videos_path(celeb_synthesis_path)
print(len(celeb_synthesis_videos_path))

real_videos_path = make_list_of_videos_path(celeb_real_path) + make_list_of_videos_path(youtube_real_path)
print(len(real_videos_path))

5639
890


In [4]:
with open("../List_of_videos/test_real.txt", 'r') as file:
    test_real_dataset = file.read().replace("/", "\\").split("\n")
test_real_dataset = [os.path.join(base_path, video_path) for video_path in test_real_dataset]
# Removing from the real_videos_path the videos from the test dataset
real_videos_path = list(set(real_videos_path) - set(test_real_dataset))

common_elements = set(test_real_dataset).intersection(set(real_videos_path))
if common_elements:
    print("There are equal elements in both lists:", common_elements)
else:
    print("No equal elements")




with open("../List_of_videos/test_fake.txt", 'r') as file:
    test_fake_dataset = file.read().replace("/", "\\").split("\n")
test_fake_dataset = [os.path.join(base_path, video_path) for video_path in test_fake_dataset]
# Removing from the celeb_synthesis_videos_path the videos from the test dataset
celeb_synthesis_videos_path = list(set(celeb_synthesis_videos_path) - set(test_fake_dataset))

common_elements = set(celeb_synthesis_videos_path).intersection(set(test_fake_dataset))
if common_elements:
    print("There are equal elements in both lists:", common_elements)
else:
    print("No equal elements")

No equal elements
No equal elements


In [5]:
def check_videos_with_fewer_frames(list_of_videos, target_number_of_frames):
    
    total_videos = len(list_of_videos)
    videos_with_less_frames = []

    for i, video_path in enumerate(list_of_videos):
        video_capture = cv2.VideoCapture(video_path)
        
        # Checking if the video can be oppened
        if not video_capture.isOpened():
            print("Não foi possivel abrir:", video_path)
        else:
            total_frames = int(cv2.VideoCapture.get(video_capture, cv2.CAP_PROP_FRAME_COUNT)) 
            if  total_frames < target_number_of_frames:
                videos_with_less_frames.append(video_path)

                video_name = video_path.split("\\")[-1]
                print(f"{i+1}/{total_videos} - {video_name} não possui o tamanho necessario - Possui {total_frames} frames")
                
    return videos_with_less_frames


# Generate train and val datasets

In [6]:
def write_list_to_file(file_path, videos_path_list):
    with open(file_path, 'w') as file:
        for string in videos_path_list:
            string = string.split('\\')[-2:]
            string = "/".join(string)
            file.write(string + '\n')
    print("Content has been written to the file successfully.")


## Real videos

In [7]:
#Remove from real_videos_path the ones with less than 24*2 frames
videos_with_less_than_24x2_frames = check_videos_with_fewer_frames(real_videos_path, 24*2)
for video in videos_with_less_than_24x2_frames:
    real_videos_path.remove(video)


# Now the list of real videos only have videos with enough frames for validation
# Since there are videos that don't have enough frames for train, they will become part of the val_real_dataset
target_validation_videos = 163
val_real_dataset = check_videos_with_fewer_frames(real_videos_path, 24*9)

# These videos will then be removed from the real_videos_path list
for video in val_real_dataset:
    real_videos_path.remove(video)

# Then a sample of videos from real_path_videos will be randomly choosen to compose the val_real_dataset
target_validation_videos -= len(val_real_dataset)
val_real_dataset += random.sample(real_videos_path, target_validation_videos)

# Finally, the train_real_dataset will be the remainder of the real_videos_path
train_real_dataset = list(set(real_videos_path) - set(val_real_dataset))

print(len(val_real_dataset))
print(len(train_real_dataset))


# Check if there are equal elements in both lists
common_elements = set(val_real_dataset).intersection(set(train_real_dataset))
if common_elements:
    print("There are equal elements in both lists:", common_elements)
else:
    write_list_to_file("../List_of_videos/val_real.txt", val_real_dataset)
    write_list_to_file("../List_of_videos/train_real.txt", train_real_dataset)

249/712 - id27_0005.mp4 não possui o tamanho necessario - Possui 1 frames
67/711 - id17_0006.mp4 não possui o tamanho necessario - Possui 201 frames
144/711 - id21_0003.mp4 não possui o tamanho necessario - Possui 178 frames
203/711 - id8_0005.mp4 não possui o tamanho necessario - Possui 215 frames
447/711 - id7_0005.mp4 não possui o tamanho necessario - Possui 161 frames
571/711 - id10_0006.mp4 não possui o tamanho necessario - Possui 172 frames
633/711 - id23_0003.mp4 não possui o tamanho necessario - Possui 186 frames
682/711 - id21_0005.mp4 não possui o tamanho necessario - Possui 165 frames
163
548
Content has been written to the file successfully.
Content has been written to the file successfully.


## Fake videos

In [8]:
# With previous investigation was possible to confirm that none of the fake videos has less than 24 frames

target_validation_videos = 326

val_fake_dataset = random.sample(celeb_synthesis_videos_path, target_validation_videos)
train_fake_dataset = list(set(celeb_synthesis_videos_path) - set(val_fake_dataset))

print(len(val_fake_dataset))
print(len(train_fake_dataset))

# Check if there are equal elements in both lists
common_elements = set(val_fake_dataset).intersection(set(train_fake_dataset))
if common_elements:
    print("There are equal elements in both lists:", common_elements)
else:
    write_list_to_file("../List_of_videos/val_fake.txt", val_fake_dataset)
    write_list_to_file("../List_of_videos/train_fake.txt", train_fake_dataset)

326
4973
Content has been written to the file successfully.
Content has been written to the file successfully.
